In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,25807
2,Huelva,Confirmados PDIA 14 días,771
3,Huelva,Tasa PDIA 14 días,"150,24260966151567"
4,Huelva,Confirmados PDIA 7 días,424
5,Huelva,Total Confirmados,26008
6,Huelva,Curados,21166
7,Huelva,Fallecidos,355


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  25807.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6748.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1844 personas en los últimos 7 días 

Un positivo PCR cada 1251 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,25807.0,424.0,771.0,513170.0,82.623692,150.242610,126.0
Condado-Campiña,7562.0,256.0,469.0,156231.0,163.859925,300.196504,63.0
Huelva-Costa,15484.0,141.0,257.0,289548.0,48.696589,88.759031,54.0
Huelva (capital),6748.0,78.0,115.0,143837.0,54.228050,79.951612,27.0
Almonte,1045.0,62.0,136.0,24507.0,252.988942,554.943486,17.0
Cartaya,1026.0,29.0,52.0,20083.0,144.400737,258.925459,11.0
Trigueros,240.0,12.0,12.0,7862.0,152.632918,152.632918,8.0
San Bartolomé de la Torre,244.0,21.0,34.0,3761.0,558.362138,904.014890,7.0
Sierra de Huelva-Andévalo Central,2338.0,22.0,30.0,67391.0,32.645309,44.516330,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Bartolomé de la Torre,244.0,21.0,34.0,3761.0,558.362138,904.014890,7.0
Rociana del Condado,385.0,30.0,61.0,7939.0,377.881345,768.358735,4.0
Alosno,151.0,2.0,23.0,3933.0,50.851767,584.795322,1.0
Almonte,1045.0,62.0,136.0,24507.0,252.988942,554.943486,17.0
Niebla,175.0,5.0,16.0,4116.0,121.477162,388.726919,0.0
Bollullos Par del Condado,686.0,35.0,53.0,14387.0,243.275179,368.388128,3.0
Condado-Campiña,7562.0,256.0,469.0,156231.0,163.859925,300.196504,63.0
Bonares,198.0,16.0,18.0,6060.0,264.026403,297.029703,2.0
Palos de la Frontera,492.0,17.0,31.0,11742.0,144.779424,264.009538,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,151.0,2.0,23.0,3933.0,50.851767,584.795322,1.0,0.086957
Paterna del Campo,120.0,1.0,7.0,3457.0,28.926815,202.487706,0.0,0.142857
Ayamonte,1247.0,4.0,14.0,21104.0,18.953753,66.338135,2.0,0.285714
Niebla,175.0,5.0,16.0,4116.0,121.477162,388.726919,0.0,0.312500
Isla Cristina,2013.0,3.0,8.0,21393.0,14.023279,37.395410,1.0,0.375000
Gibraleón,550.0,10.0,22.0,12737.0,78.511423,172.725132,4.0,0.454545
Almonte,1045.0,62.0,136.0,24507.0,252.988942,554.943486,17.0,0.455882
Rociana del Condado,385.0,30.0,61.0,7939.0,377.881345,768.358735,4.0,0.491803
Valverde del Camino,431.0,4.0,8.0,12750.0,31.372549,62.745098,1.0,0.500000
